In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.tri as mtri
from tqdm import tqdm_notebook
import odr
import odr.ed_quantum as ed
from odr.model_operational import OperationalNetwork
from odr.data_handler import Dataset

## Generate or load data

In [ ]:
data = ed.create_dataset_mixed(int(1e5))
data.save('quantum_mixed_100k')

In [ ]:
data = Dataset.load('quantum_mixed_100k')

In [ ]:
td, vd = data.train_val_separation(0.05)

## Train model

In [ ]:
gamma = 2e-5
model = OperationalNetwork(encoder_num=1, decoder_num=3, input_sizes=[75], latent_sizes=[20], 
            question_sizes=[75, 75, 75], answer_sizes=[1, 1, 1],
            encoder_num_units=[500, 250], decoder_num_units=[500, 100], name='quantum_mixed')

model.train(1000, 512, 5e-4, td, vd, test_step=1, reg_loss_factor=5e-6, gamma=gamma, progress_bar=tqdm_notebook)
model.save(model.name)

## Load pre-trained model and plot latent layer

In [ ]:
model = OperationalNetwork.from_saved('quantum_mixed_states')

In [ ]:
from scipy.linalg import sqrtm
from scipy.stats import unitary_group
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1.j], [1.j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])
swap_gate = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])

def vec_to_state(vec):
    return 1/2.*(np.eye(2) + vec[0] * sigma_x + vec[1] * sigma_y + vec[2] * sigma_z)

def gen_rand_mixed_with_fixed_axis(fix_qubit, fix_axis):
    r = np.random.rand()
    phi = 2*np.pi * np.random.rand()
    single_qubit_vec = np.roll(np.array([0, r*np.cos(phi), r*np.sin(phi)]), fix_axis)
    single_qubit_state = vec_to_state(single_qubit_vec)
    purification = np.dot(np.kron(sqrtm(single_qubit_state), np.eye(2)), np.array([1, 0, 0, 1]))
    U = unitary_group.rvs(8)
    randomized_state = np.dot(np.kron(np.eye(2), U), np.kron(purification, np.array([1, 0, 0, 0])))
    randomized_state = np.outer(randomized_state, randomized_state.conj())
    red_rand_state = np.trace(randomized_state.reshape(4, 4, 4, 4), axis1=1, axis2=3)
    if fix_qubit == 1:
        red_rand_state = swap_gate.dot(red_rand_state.dot(swap_gate))
    return red_rand_state

def red_state_coords(joint_state, qubit_no):
    axes = [0, 2] if qubit_no == 1 else [1, 3]
    red_state = np.trace(joint_state.reshape(2, 2, 2, 2), axis1=axes[0], axis2=axes[1])
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_y = np.array([[0, -1.j], [1.j, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])
    
    return [np.real(np.trace(np.dot(sigma_x, red_state))), 
            np.real(np.trace(np.dot(sigma_y, red_state))), 
            np.real(np.trace(np.dot(sigma_z, red_state)))]

In [ ]:
def plot3D_from_data(model, dataset_both, x_data_both, y_data_both, zs=None, labels_xy=['x', 'y'], zlim=None):
    layer = model.full_latent

    noise_levels = model.run(dataset_both[0], model.select_logs)
    selected_latent = np.sort(np.concatenate([np.where(noise_levels[0] < 0)[0], 
                                             np.where(noise_levels[1] < 0)[0]]))
    print(selected_latent)
    zs_both = [model.run(dataset_both[0], layer), model.run(dataset_both[1], layer)]
    
    fig = plt.figure(figsize=(20, 7))
    for counter, latent_index in enumerate(selected_latent):
        for qubit_no in range(2):
            # Set up triangulation
            tri = mtri.Triangulation(x_data_both[qubit_no], y_data_both[qubit_no])
            points = np.dstack([x_data_both[qubit_no][tri.triangles], y_data_both[qubit_no][tri.triangles]])
            edge_length = np.empty(points.shape[0])
            for i in range(len(edge_length)):
                p = points[i]
                edge_length[i] = np.linalg.norm(p[0] - p[1]) + np.linalg.norm(p[0] - p[2]) + np.linalg.norm(p[1] - p[2])
            threshold = 5.
            mask = np.where(edge_length > threshold, True, False)
            tri.set_mask(mask)

            ax = fig.add_subplot(2, 6, counter + qubit_no*6 + 1, projection='3d')
            ax.tick_params(labelsize=14)
            ax.azim=-115
            z_min = np.min(np.stack([zs_both[0][:, latent_index], zs_both[1][:, latent_index]]))
            z_max = np.max(np.stack([zs_both[0][:, latent_index], zs_both[1][:, latent_index]]))
            cmap = cm.inferno if noise_levels[qubit_no][latent_index] < 0 else cm.Greys
            ax.plot_trisurf(tri, zs_both[qubit_no][:, latent_index], cmap=cmap, vmin=z_min, vmax=z_max)
            ax.set_xlabel(labels_xy[0], fontsize=18)
            ax.set_ylabel(labels_xy[1], fontsize=18)
            ax.set_zlim([z_min, z_max])
            ax.set_xticks([-1,0,1])
            ax.set_yticks([-1,0,1])
            ax.set_zticks([-1,0,1])
    fig.tight_layout()
    return fig

def plot_quantum(model, joint_ref_states, fix_axis, n=1000):
    xx_both = []
    yy_both = []
    zz_both = []
    viz_dataset_both = []
    
    for qubit_no in [0, 1]:
        joint_states = np.array([gen_rand_mixed_with_fixed_axis(qubit_no, fix_axis) for _ in range(n)])
        viz_dataset = ed.create_dataset_mixed(None, manual_ref_states=[joint_states, joint_ref_states])

        xx, yy, zz = [], [], []
        for joint_state in viz_dataset.hidden_states[0][:n]:
            coords = red_state_coords(joint_state, qubit_no)
            xx.append(coords[0])
            yy.append(coords[1])
            zz.append(coords[2])

        xx_both.append(np.array(xx))
        yy_both.append(np.array(yy))
        zz_both.append(np.array(zz))
        viz_dataset_both.append(viz_dataset)
    
    if fix_axis == 0:
        return plot3D_from_data(model, viz_dataset_both, yy_both, zz_both, labels_xy=[r'$y$', r'$z$'])
    elif fix_axis == 1:
        return plot3D_from_data(model, viz_dataset_both, xx_both, zz_both, labels_xy=[r'$x$', r'$z$'])
    elif fix_axis == 2:
        return plot3D_from_data(model, viz_dataset_both, xx_both, yy_both, labels_xy=[r'$x$', r'$y$'])
    

In [ ]:
# To make the plots, we require the global parameters of the dataset.
# If you would like to plot the pre-trained model, use:
import pickle
with open('pretrained_ref_states.pkl', 'rb') as f:
    joint_ref_states = pickle.load(f)

# If you would like to plot a model that you trained on your own dataset, use:
# joint_ref_states = data.global_params[0]

In [ ]:
%matplotlib tk
fig = plot_quantum(model, joint_ref_states, fix_axis=1, n=1000)

In [ ]:
fig = plot_quantum(model, joint_ref_states, fix_axis=0, n=1000)